In [53]:
with open("C:/Users/yuvra/OneDrive/Desktop/LLM FROM SCRATCH/LLM-FROM-SCRATCH/chapters/chapter_01/text.txt", "r") as f:
    text = f.read()
    print("Length of txt file", len(text))

Length of txt file 2736


In [54]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([.?,"]|--|\s)',text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?!()\'])', r'\1', text)
        return text
        

In [55]:
import re

result = re.split(r'(,)|(\s)', text)
print("split text", result)
for items in result:
    if items == None:
        print(items)
        result.remove(items)

print("splitted and removed none values: ", result)

# Removing empty strings from the list
result = [items for items in result if items.strip()]
print("removed empty strings from list: ", result)
print("lemgth of list: ", len(result))

#removing duplicates and sorting the list using set
all_words = sorted(set(result))
print("removed duplicates and sorted the list: ", all_words)
print("length of list now: ", len(all_words))

print("type of all_words", type(all_words)) #all_words is a list

# Creating a vocabulary dictionary from the list of words
vocabulary = {token:integer for integer, token in enumerate(all_words)}
vocabulary[228] = "."
print(vocabulary)
print(len(vocabulary.items()))

split text ['Hello', ',', None, '', None, '\n', '', None, '\n', 'About', None, ' ', 'your', None, ' ', 'first', None, ' ', 'question', ',', None, '', None, ' ', 'I', None, ' ', 'also', None, ' ', 'just', None, ' ', 'started', None, ' ', 'to', None, ' ', 'CS50w', None, ' ', 'and', None, ' ', 'tried', None, ' ', 'to', None, ' ', 'use', None, ' ', 'git', None, ' ', 'to', None, ' ', 'submit', None, ' ', 'project', None, ' ', '0', None, ' ', 'like', None, ' ', 'you.', None, ' ', 'And', None, ' ', 'after', None, ' ', 'some', None, ' ', 'struggle', ',', None, '', None, ' ', 'I', None, ' ', 'managed', None, ' ', 'to', None, ' ', 'do', None, ' ', 'it.', None, ' ', 'I', None, ' ', 'will', None, ' ', 'try', None, ' ', 'to', None, ' ', 'help', None, ' ', 'you', None, ' ', 'about', None, ' ', 'it.', None, '\n', '', None, '\n', '1-', None, ' ', 'Make', None, ' ', 'sure', None, ' ', 'about', None, ' ', 'your', None, ' ', 'file', None, ' ', 'system.', None, ' ', 'What', None, ' ', 'I', None, ' ', 'mea

In [56]:
tokenizer = SimpleTokenizerV1(vocabulary)
text = """About your first question, I also just started to CS50w and tried to use git to submit project 0 like you And after some struggle, I managed to do it I will try to help you about it

1- Make sure about your file What I mean is, don't create new folders inside your project to wrap actual project files

Ex: For project:0 which is search, make sure you are inside search/ folder and all your html+css inside of it
"""
ids = tokenizer.encode(text)
print(ids)

[18, 227, 109, 169, 8, 29, 58, 135, 183, 200, 21, 59, 202, 200, 206, 115, 200, 187, 165, 10, 139, 224, 20, 56, 180, 186, 8, 29, 145, 200, 98, 132, 29, 219, 203, 200, 117, 224, 51, 132, 11, 35, 189, 51, 227, 105, 40, 29, 150, 131, 8, 99, 90, 154, 111, 130, 227, 165, 200, 223, 52, 165, 106, 24, 25, 167, 217, 131, 174, 8, 144, 189, 224, 61, 130, 175, 110, 59, 57, 227, 120, 130, 158, 132]


In [57]:
all_words = sorted(list(set(result)))
all_words.extend(["|<unk>|", "|<endoftext>|"])
vocab = {token:integer for integer, token in enumerate(all_words)}

print(len(vocab.items()))

230


In [58]:
context_size = 4

# with open("C:/Users/yuvra/OneDrive/Desktop/LLM FROM SCRATCH/LLM-FROM-SCRATCH/chapters/chapter_01/text.txt", "r") as f:
#     new_text = f.read()

enc_text = tokenizer.encode(text)

for i in range(1, context_size+1):
    x = enc_text[:i]
    y = enc_text[i: context_size+1]
    print(f"{x}")
    print(f"  {y}")

for i in range(1, context_size+1):
 context = enc_text[:i]
 desired = enc_text[i]
 print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

[18]
  [227, 109, 169, 8]
[18, 227]
  [109, 169, 8]
[18, 227, 109]
  [169, 8]
[18, 227, 109, 169]
  [8]
About ----> your
About your ----> first
About your first ----> question
About your first question ----> ,


In [59]:
import torch
from torch.utils.data import Dataset, DataLoader

class GPTtokenizer:
    def __init__(self, txt, tokenizer, maximum_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt)

        for i in range(0, len(token_ids) - maximum_length, stride):
            input_chunk = token_ids[i:i + maximum_length]
            target_chunks = token_ids[i+1: maximum_length+1]

            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunks))

    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

print(torch.__version__)
from importlib.metadata import version

print(version("torch"))


if torch.cuda.is_available():
    print("Yes")
else:
    print("No")

2.7.1+cpu
2.7.1+cpu
No
